In [43]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

In [ ]:
# 최적화 공식

def optimize(region):
    try:
        # Create a new model
        m = gp.Model("milp")

        # Create variables
        for i in range(1, len(region)+1):
            globals()['x%d'%i] = m.addVar(vtype=GRB.INTEGER, name="x%d"%i)
        for i in range(0, len(region)):
            globals()['y%d'%(i+1)] = region.loc[i, '충전소 수']
            globals()['n%d'%(i+1)] = region.loc[i, 'population']
            
        # Standard Value Setting
        p = 1000000/1500
        # Set objective
        obj = None
        for i in range(1, len(region)+1):
            obj += globals()['x%d'%i] * globals()['n%d'%i]
        m.setObjective(obj, GRB.MAXIMIZE)

        const = globals()['x1']
        for i in range(2, len(region)+1):
            const +=  globals()['x%d'%i]

        # Add constraint: x + 2 y + 3 z <= 4
        m.addConstr(1400 * const <= 269000, "c0")

        # Add constraint: x + y >= 1
        for i in range(1, len(region)+1):
            if globals()['n%d'%i]/p - globals()['y%d'%i] > 0:
                m.addConstr(globals()['x%d'%i] <= globals()['n%d'%i]/p - globals()['y%d'%i], "c_%d"%i)
            else:
                m.addConstr(globals()['x%d'%i] <= 0)

        # Optimize model
        m.optimize()

        for v in m.getVars():
            print('%s %g' % (v.varName, v.x))

        print('Obj: %g' % m.objVal)
        
        return m

    except gp.GurobiError as e:
        print('Error code ' + str(e.errno) + ': ' + str(e))
        return None

    except AttributeError:
        print('Encountered an attribute error')
        return None

In [44]:
# Data Load

df = pd.read_csv('EV_2020.csv', encoding='cp949')
df.head()
df = df.groupby('일시').sum().mean()
df_noon = df.iloc[9:18]
df_night = df.drop(index=df_noon.index)

df_noon.mean()
df_night.mean()
Noon = df_noon.mean() /(df_noon.mean() + df_night.mean())
Night = df_night.mean() /(df_noon.mean() + df_night.mean())

# 전기차량 낮/밤 가중치
print((Noon, Night))

In [46]:
# 낮/밤 충전소 부하량

region = pd.read_csv('Region_Charger.csv')
region.rename(columns={'Unnamed: 0':'loc'}, inplace=True)
region.head(10)

,Unnamed: 0,noon,night,충전소 수
0,강동구,260045.986690,295823.714978,285
1,송파구,469660.421729,471872.300279,540
2,강남구,664934.519079,465326.925569,660
3,서초구,450708.846949,342209.423613,751
4,관악구,278161.474542,336942.093492,254
5,동작구,223512.352788,257178.359332,307
6,영등포구,359691.293567,288125.145915,421
7,금천구,165505.493276,139008.257913,365
8,구로구,245558.027474,247917.425536,495
9,강서구,315460.667253,343064.182929,609


In [47]:
# 낮/밤 충전소 + 인구 데이터

region.loc[:,'population'] = region.noon * Noon + region.night * Night
region.loc[:, 'proportion'] = region.population/ region.iloc[:,3]
region = region.drop(columns = ['noon', 'night'], axis = 1)

In [48]:
region.head(10)

,Unnamed: 0,충전소 수,population,proportion
0,강동구,285,271240.966191,951.722688
1,송파구,540,470352.526460,871.023197
2,강남구,660,602476.587164,912.843314
3,서초구,751,416758.988246,554.938733
4,관악구,254,296554.141032,1167.535988
5,동작구,307,234046.566981,762.366668
6,영등포구,421,337297.989289,801.182872
7,금천구,365,157214.413297,430.724420
8,구로구,495,246296.291589,497.568266
9,강서구,609,324097.906310,532.180470


In [9]:
project_year = 5

for i in range(project_year):
    m = optimize(region)
    if m == None:
        print('최적화 실패')
        break;
    lst = []
    for e in m.getVars() :
        lst.append(e.x)
    if i == 0:
        data = pd.DataFrame(lst, columns = ['Year{}'.format(i)], index = region.iloc[:,0])
    else:
        data = pd.concat([data, pd.DataFrame(lst, columns = ['Year{}'.format(i)], index = region.iloc[:,0])], axis=1)
    region.iloc[:,1] = region.iloc[:,1] + [int(e.x) for e in m.getVars()]
data

Academic license - for non-commercial use only - expires 2022-07-23
Using license file C:\Users\USER\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 26 rows, 25 columns and 50 nonzeros
Model fingerprint: 0x88ecb4c8
Variable types: 0 continuous, 25 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+05, 6e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+05]
Found heuristic solution: objective 6.621519e+07
Presolve removed 26 rows and 25 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 1.15676e+08 6.62152e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.156755047354e+08, best bound 1.156755047354e+08, gap 0.0000%
x1 -0
x2 -0
x3 192
x4 0
x5 -0
x6 -0
x